In [1]:
import sys
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def listing_invalid(price):
    # check if price is free
    if price == "Free":
        return True
    price = price.lstrip('$')
    price = price.replace(',', '')
    
    # check if price is $123, $1234, ...
    price = float(price)
    if price == 1 or price == 123 or price == 1234 or price == 12345 or \
    price == 123456 or price == 1234567 or price == 12345678 or price == 123456789:
        return True
    else:
        return False

In [3]:
def search_scrape_fb(query):

    # handling search errors
    while True:
        try:
            chrome_options = Options()
            chrome_options.add_experimental_option("detach", True)
            driver = webdriver.Chrome(options = chrome_options)
            PAGE_LOAD_BUFFER = 1
            driver.get("https://www.facebook.com/marketplace/")
            driver.maximize_window()
            driver.find_element(By.CSS_SELECTOR, "h1[class='x1heor9g x1qlqyl8 x1pd3egz x1a2a7pz']")

            # specify wait = 5 seconds
            wait = WebDriverWait(driver, 5)

            # change location
            location_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='x1iyjqo2']/span")))
            location_button.click()
            locate_me_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[aria-label='Marketplace geolocation picker']")))
            locate_me_button.click()
            time.sleep(PAGE_LOAD_BUFFER)
            apply_location_button = driver.find_element(By.CSS_SELECTOR, "div[aria-label='Apply']")
            apply_location_button.click()
            time.sleep(PAGE_LOAD_BUFFER)

            # search bar
            search_bar = driver.find_element(By.CSS_SELECTOR, "input[aria-label='Search Marketplace']")
            search_bar.send_keys(query)
            search_bar.send_keys(Keys.ENTER)
            time.sleep(PAGE_LOAD_BUFFER)

            # filter last 24 hours
            date_listed_button = driver.find_elements(By.CSS_SELECTOR, "div[class='x9f619 x78zum5 xl56j7k xs9asl8 xurb0ha x1iorvi4 xh8yej3']")[3]
            date_listed_button.click()
            last_24_hrs_button = driver.find_elements(By.CSS_SELECTOR, "div[class='x6s0dn4 xkh2ocl x1q0q8m5 x1qhh985 xu3j5b3 xcfux6l x26u7qi xm0m39n x13fuv20 x972fbf x9f619 x78zum5 x1q0g3np x1iyjqo2 xs83m0k x1qughib xat24cr x11i5rnm x1mh8g0r xdj266r x2lwn1j xeuugli x18d9i69 x4uap5 xkhd6sd xexx8yu x1n2onr6 x1ja2u2z']")[13]
            last_24_hrs_button.click()
            time.sleep(PAGE_LOAD_BUFFER)

#             # scroll to bottom
#             SCROLL_LOAD_BUFFER = 1
#             last_height = driver.execute_script("return document.body.scrollHeight")
#             while True:
#                 driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#                 time.sleep(SCROLL_LOAD_BUFFER)
#                 new_height = driver.execute_script("return document.body.scrollHeight")
#                 if new_height == last_height:
#                     break
#                 last_height = new_height 
        except:
            print("Error loading Facebook Marketplace, trying again.")
            driver.quit()
        else:
            break
            
    # find titles + prices + URL
    titles = driver.find_elements(By.CSS_SELECTOR, "span[class='x1lliihq x6ikm8r x10wlt62 x1n2onr6']")
    prices = driver.find_elements(By.CSS_SELECTOR, "span[class='x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u']")
    urls = driver.find_elements(By.CSS_SELECTOR, "a[class='x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv']")
    
    # listing information
    list = []
    num_listings = len(titles)

    for i in range(num_listings):
        if listing_invalid(prices[i].text):
            continue
        title = titles[i].text
        price = prices[i].text.lstrip('$')
        price = float(price.replace(',', ''))
        url = urls[i].get_attribute("href")
        list.append((title, price, url))

    return list